In [ ]:
import pandas as pd
import os
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import matplotlib
from matplotlib.dates import DateFormatter
import matplotlib.dates as md
import datetime as dt
from pivottablejs import pivot_ui
from IPython.display import HTML
import qgrid
import ipywidgets as widgets

In [ ]:
def time_to_int(dateobj):
    total = int(dateobj.strftime('%S'))
    total += int(dateobj.strftime('%M')) * 60
    total += int(dateobj.strftime('%H')) * 60 * 60
    total += (int(dateobj.strftime('%j')) - 1) * 60 * 60 * 24
    total += (int(dateobj.strftime('%Y')) - 1970) * 60 * 60 * 24 * 365
    return total

def myfunc(x):
    return slope * x + intercept

In [ ]:
db_filename = 'todo.db'

db_is_new = not os.path.exists(db_filename)

conn = sqlite3.connect(db_filename)

if db_is_new:
    print('Need to create schema')
else:
    print('Database exists, assume schema does, too.')

conn.close()

In [ ]:
prices = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="SP1500_Sectors_Indexes_Prices")

In [ ]:
screener = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="Screener")

In [ ]:
stock_sectors = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="stock_sectors")
stock_sectors.set_index(stock_sectors.columns[0],inplace=True)
stock_sectors.columns = ['Symbol']

In [ ]:
stock_indexes = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="SP1500")
stock_indexes.drop(stock_indexes.columns[0], axis=1, inplace=True)
stock_indexes.set_index(stock_indexes.columns[0],inplace=True)
stock_indexes

In [ ]:
spdr_indexes = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="spdr_indexes")
#vanguard_sectors.drop(vanguard_sectors.columns[0], axis=1, inplace=True)
spdr_indexes.set_index(spdr_indexes.columns[0],inplace=True)
spdr_indexes.columns=['Symbol']
spdr_indexes

In [ ]:
vanguard_sectors = pd.read_excel('SP1500_screener_multiple.xlsx',sheet_name="vanguard_sectors")
#vanguard_sectors.drop(vanguard_sectors.columns[0], axis=1, inplace=True)
vanguard_sectors.set_index(vanguard_sectors.columns[0],inplace=True)
vanguard_sectors.columns=['Symbol']
vanguard_sectors

In [ ]:
sector_dataset = pd.DataFrame()

for v in vanguard_sectors['Symbol']:
    subset = prices[prices['Symbol']==v].set_index('Date')[['Adj Close']].asfreq('Q').pct_change()
    
    plt.plot(subset)
    subset['Symbol'] = v
    sector_dataset = pd.concat([sector_dataset,subset],axis=0)
    #plt.xticks(np.array(subset.index)[np.arange(0,len(subset.index),int(len(subset.index)))], rotation=45)
    plt.xticks(subset.index, rotation=45)
plt.legend(vanguard_sectors['Symbol'].index,loc=2)    
plt.show()

d_ = []
for d in sector_dataset.reset_index()['Date']:
    d_.append(d.strftime('%Y-%m-%d'))


In [ ]:

sector_dataset['Date'] = d_

sector_dataset = sector_dataset.reset_index(drop=True).set_index('Date')
sector_dataset_pivot = sector_dataset.reset_index().pivot_table(values=['Adj Close'], index=['Symbol'],columns=['Date'])

sector_dataset_pivot.columns = sector_dataset_pivot.columns.droplevel(0)
sector_dataset_pivot

In [ ]:
qgrid.show_grid(sector_dataset_pivot)

In [ ]:
for v in spdr_indexes['Symbol']:
    subset = prices[prices['Symbol']==v].set_index('Date')
    plt.plot(subset[['Adj Close']].asfreq('Q').pct_change())
    plt.xticks(np.array(subset.index)[np.arange(0,len(subset.index),int(len(subset.index)/10))], rotation=45)
plt.legend(spdr_indexes['Symbol'].index)    
plt.show()

In [ ]:
screener['Symbol']

In [ ]:
threshold = .75

In [ ]:
risk_trend_threshold = screener[['risk_trend']].quantile(q=[threshold])
risk_screened = screener[screener['risk_trend']>risk_trend_threshold['risk_trend'].values[0]]['Symbol'].values

In [ ]:
risk_screened

In [ ]:
screener.columns

In [ ]:
above_1yearSDev = screener[screener['latest_p']>screener['mean_1y']+screener['std_1y']]['Symbol'].values
above_2yearSDev = screener[screener['latest_p']>screener['mean_2y']+screener['std_2y']]['Symbol'].values
above_yearSDev = np.intersect1d(above_1yearSDev,above_2yearSDev)

In [ ]:
#above_yearSDev

In [ ]:
above_2year_t = screener[screener['pct_2yr']>threshold]['Symbol'].values
above_1year_t = screener[screener['pct_1yr']>threshold]['Symbol'].values
above_years_t = np.intersect1d(above_2year_t,above_1year_t)

In [ ]:
#above_years_t

In [ ]:
above_combined = np.intersect1d(above_yearSDev,above_years_t)

In [ ]:
#above_combined

In [ ]:
screened_final_set = np.intersect1d(risk_screened,above_combined)

In [ ]:
#screened_final_set = risk_screened
screened_final_set

In [ ]:
#pivot_ui(prices, outfile_path='pivottablejs.html')
#HTML('pivottablejs.html')

In [ ]:
for s in screened_final_set:
    print(s)
    subset=prices[prices['Symbol']==s]
    
    dates = []
    for t in subset['Date'].values:
        d = pd.Timestamp(t).strftime('%Y-%m-%d')
        dates.append(d)
        dto = datetime.strptime(d, '%Y-%m-%d').date()

    old_ordinal = [datetime.strptime(i, '%Y-%m-%d').toordinal() for i in dates]
    new_ordinal = old_ordinal + md.date2num(np.datetime64('0000-12-31'))
    x = new_ordinal
    y= subset['Adj Close']

    slope, intercept, r, p, std_err = stats.linregress(x, y)

    mymodel = list(map(myfunc, x))

    fig, ax = plt.subplots()
    
    plt.scatter(x, y)
    plt.scatter(x, subset['trailing_1yr_max'])
    plt.scatter(x, subset['trailing_1yr_min'])
    
    #plt.scatter(x, subset['supply_trend'])
    plt.scatter(x, subset['supply_trend_1yr'])
    print("supply trend:",screener.loc[np.where(screener.set_index('Symbol').index==s)[0][0]]['vol_30d_2yr'])
    print("Sector:", stock_sectors.iloc[np.where(np.array(stock_sectors.index)==s)[0][0]][0])
    print("Index:", stock_indexes.iloc[np.where(np.array(stock_indexes.index)==s)[0][0]][0])
    
    #print("Sector:",screener.loc[np.where(screener.set_index('Symbol').index==s)[0][0]]['Sector'])
    #print("Sector:",screener.loc[np.where(screener.set_index('Symbol').index==s)[0][0]]['vol_30d_2yr'])

    plt.plot(x, mymodel)
    
    l = matplotlib.dates.AutoDateLocator()
    f = matplotlib.dates.ConciseDateFormatter(l)    
    
    ax.set_xticklabels(ax.get_xticks(), rotation = 45)
    
    locator = matplotlib.dates.AutoDateLocator()
    formatter = matplotlib.dates.ConciseDateFormatter(locator)

    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    
    date_form = DateFormatter("%Y-%m-%d")
    ax.xaxis.set_major_formatter(date_form)
    plt.show()